# LoadinG The Data

In [32]:
%matplotlib inline
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import math
from sklearn import metrics

#using the preprocess Sqlite Table to read the  data
con = sqlite3.connect('./amazon-fine-food-reviews/finalsqlite')

#Filtering only positive review
positive_review = pd.read_sql_query("""
SELECT * FROM Reviews Where Score == 'positive'
""", con)

#Extracting 2000 Positive review Randomly
positive_smaller_review_set = positive_review.sample(100)
print(positive_smaller_review_set.shape)

#Filtering only negative review
negative_review = pd.read_sql_query("""
SELECT * FROM Reviews Where Score == 'negative'
""", con)

#Extracing 2000 random negative review
negative_smaller_review_set = negative_review.sample(100)
print(negative_smaller_review_set.shape)

#Concanating Negative and Positive review into one dataframe
frames = [positive_smaller_review_set, negative_smaller_review_set]
final = pd.concat(frames)

#Sort the dataframe data into the increasing Time order
final = final.sort_values(by=['Time'])
print(final.head)
print(final.shape)


(100, 12)
(100, 12)
<bound method NDFrame.head of          index      Id   ProductId          UserId  \
4447     16085   17572  B0000GH6UQ  A2Q2IWWX5MLU9C   
3770    510873  552381  B0007OPW5C  A1BDOVKT96RTYJ   
1942    249632  270675  B0000AH9XL  A1DOP5WV948LA0   
10206   243963  264554  B000FJJ2IC   AR2S1RYXQ8ST3   
7931    307661  333198  B000EQYSOY  A1Y7JCTEBJC7K1   
16543   115798  125585  B000P3T4HW  A24BKYGSAPU85J   
2937    391781  423661  B0004M0DDS  A1TV3ECUYVVIXP   
88820    21312   23333  B000MPFJAO  A11V9GP93SA9MC   
27619   132835  144195  B000BBY7YS  A1GVGQGWK8HI03   
14788   412946  446608  B000LKZEP0  A1ITT6740FONU4   
9665    347922  376346  B000FEFECA  A33CMQWETSAY0X   
14652    72682   79112  B000LKYNLG  A2DMK3REP7LEGI   
54121   352996  381825  B000FBM494   A2624W1VFNDLI   
64275   502065  542819  B000GB0TMW  A1FF072J0HASUR   
102505  116216  126023  B000SVFAHE  A3M4CNFB5OF7S8   
88430    54167   58800  B000MIFS3Y   ASPZBFGF4AS8H   
17465   265071  287333  B000SAPV

In [33]:
# Function to  find accuracy and optimal alpha  on given vector

def Naive_Bayes_accuracy(X_train, y_train):
    
    #List of alpha values to check for optimal alpha value
    alphas =  np.array([0.00001, 0.00003, 0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10])

    cv_scores = []
     
    # perform 10 fold cross validation
    for k in alphas:
        clf = MultinomialNB(alpha = k )
        scores = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        cv_scores.append(scores.mean())

    # changing to misclassification error
    MSE = [1 - x for x in cv_scores]
    print(MSE)
    # determining best alpha
    optimal_alpha = alphas[MSE.index(min(MSE))]
    print('\nThe optimal value of alpha is %d.' % optimal_alpha)
    
    # Feature Importance
    
    return optimal_alpha


# Bag OF word(BOW)

In [68]:
#BOW of preprocess text
count_vect = CountVectorizer()
final_counts_preprocess_text = count_vect.fit_transform(final['CleanedText'].values)

print("Shape of vector of Preprocess review(text) after BOW(unigram)",final_counts_preprocess_text.get_shape())

Shape of vector of Preprocess review(text) after BOW(unigram) (200, 1833)


# Train and Test data Split 

In [69]:
# create design matrix X and target vector y

X = np.array(final_counts_preprocess_text.toarray()[:,:])  
y = np.array(final['Score'].values)
print(X.shape)
print(y.shape)
# Calculating the train and test size and dividing the data into train and test chunks into the 70 - 30 ratio
train_size = math.floor(X.shape[0] * 0.7)

#split the data set into train and test
X_train = X[: train_size ,:]
X_test  = X[train_size : , :]  
y_train = y[: train_size ]
y_test =  y[train_size : ]

(200, 1833)
(200,)


 # Naive Bayes on BOW vector

In [70]:
# Using Naive_Bayes_accuracy Function finding optimal alpha 
from sklearn.metrics import accuracy_score
optimal_alpha =  Naive_Bayes_accuracy(X_train, y_train)

#calculate the accuracy on Test data

clf = MultinomialNB(alpha = optimal_alpha)

# fitting the model
nb = clf.fit(X_train, y_train)

# Predicted Probability of different classes in Ordering
print(clf.classes_)

# Calculate the log probability of  the test data
log_proba = clf.predict_log_proba(X_test)

#Calculate the Class 
y_pred = np.argmax(log_proba, axis=1) 

#Converting y_test array from positive/Neagtive to  1/0 array
y_test[np.where(y_test == 'positive')] = 1
y_test[np.where(y_test == 'negative')] = 0

# Features After BOW
count_vect = count_vect.get_feature_names()

#log probability. for words with higher probability for both positive and negative reviews
word_probability = np.argmax(nb.feature_log_prob_, axis=1) 
print("log probability for words with higher probability for negative reviews is  ", count_vect[word_probability[0]])
print("log probability for words with higher probability for positive reviews is ", count_vect[word_probability[1]])

print(y_test)
print(y_pred)
print(accuracy_score(y_test, y_pred))


# Accuracy , precision, recall, f1 score, confusion matrix for Optimal value of alpha
# print("\nThe accuracy of the Naive Bayes classifier on Bag Of Words for optimal_alpha is ",metrics.accuracy_score(y_test, y_pred))
# print("\nThe confusion_matrix of the Naive Bayes classifier on Bag Of Words for optimal_alpha is ",metrics.confusion_matrix(y_test, y_pred))
# print("\nThe recall_score of the Naive Bayes classifier on Bag Of Words for optimal_alpha is ",metrics.recall_score(y_test, y_pred))
# print("\nThe precision_score of the Naive Bayes classifier on Bag Of Words for optimal_alpha is ",metrics.precision_score(y_test, y_pred))
# print("\nThe f1_score of the Naive Bayes classifier on Bag Of Words for optimal_alpha is ",metrics.f1_score(y_test, y_pred))


[0.2695604395604395, 0.2695604395604395, 0.2695604395604395, 0.2695604395604395, 0.2624175824175824, 0.2624175824175824, 0.2624175824175824, 0.2552747252747253, 0.24194139194139197, 0.27208791208791205, 0.28589743589743577, 0.29468864468864464, 0.3017582417582416]

The optimal value of alpha is 0.
['negative' 'positive']
log probability for words with higher probability for negative reviews is   tast
log probability for words with higher probability for positive reviews is  like
[0 1 1 0 0 0 1 1 1 1 0 1 0 1 1 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0
 1 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1]
[0 0 1 0 0 0 0 1 1 0 1 0 1 1 0 1 1 0 1 0 1 0 0 1 0 0 0 1 0 1 0 1 1 0 1 1 0
 0 1 0 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1]


ValueError: Classification metrics can't handle a mix of unknown and binary targets

# Tf-Idf

In [11]:
#tf-idf of preprocess text
tf_idf_vect  = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf_preprocess_text =  tf_idf_vect.fit_transform(final['CleanedText'].values)

#Shape of  vectors
print("Shape of vector of Preprocess review(text) after Tf-Idf",final_tf_idf_preprocess_text.get_shape())

Shape of vector of Preprocess review(text) after Tf-Idf (100, 7274)


# Train and Test data split

In [ ]:
# create design matrix X and target vector y

X = np.array(final_tf_idf_preprocess_text.toarray()[:,:])  
y = np.array(final['Score'].values)
print(X.shape)
print(y.shape)
# Calculating the train and test size and dividing the data into train and test chunks into the 70 - 30 ratio
train_size = math.floor(X.shape[0] * 0.7)

#split the data set into train and test
X_train = X[: train_size ,:]
X_test  = X[train_size : , :]  
y_train = y[: train_size ]
y_test =  y[train_size : ]

# Naive Bayes on TF-IDF vector 

In [13]:
# Using Naive_Bayes_accuracy Function finding optimal alpha 
from sklearn.metrics import accuracy_score
optimal_alpha =  Naive_Bayes_accuracy(X_train, y_train)

#calculate the accuracy on Test data

clf = MultinomialNB(alpha = optimal_alpha)

# fitting the model
nb = clf.fit(X_train, y_train)

# Predicted Probability of different classes in Ordering
print(clf.classes_)

# Calculate the log probability of  the test data
log_proba = clf.predict_log_proba(X_test)

#Calculate the Class 
y_pred = np.argmax(log_proba, axis=1) 

#Converting y_test array from positive/Neagtive to  1/0 array
y_test[np.where(y_test == 'positive')] = 1
y_test[np.where(y_test == 'negative')] = 0

# Features After Naive Bayes
tf_idf_vect_features = tf_idf_vect.get_feature_names()

#log probability. for words with higher probability for both positive and negative reviews
word_probability = np.argmax(nb.feature_log_prob_, axis=1) 
print("log probability for words with higher probability for negative reviews is  ", tf_idf_vect_features[word_probability[0]])
print("log probability for words with higher probability for positive reviews is ", tf_idf_vect_features[word_probability[1]])

print(y_test)
print(y_pred)
print(accuracy_score(y_test, y_pred))


# Accuracy , precision, recall, f1 score, confusion matrix for Optimal value of alpha
# print("\nThe accuracy of the Naive Bayes classifier on Bag Of Words for optimal_alpha is ",metrics.accuracy_score(y_test, y_pred))
# print("\nThe confusion_matrix of the Naive Bayes classifier on Bag Of Words for optimal_alpha is ",metrics.confusion_matrix(y_test, y_pred))
# print("\nThe recall_score of the Naive Bayes classifier on Bag Of Words for optimal_alpha is ",metrics.recall_score(y_test, y_pred))
# print("\nThe precision_score of the Naive Bayes classifier on Bag Of Words for optimal_alpha is ",metrics.precision_score(y_test, y_pred))
# print("\nThe f1_score of the Naive Bayes classifier on Bag Of Words for optimal_alpha is ",metrics.f1_score(y_test, y_pred))


brute
[0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5267857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.4708333333333333, 0.5, 0.5125, 0.4976190476190475, 0.4976190476190476]

The optimal number of neighbors is 21.

The accuracy of the knn classifier on Tf-Idf using Brute Force approach for k = 21 is 53.333333%
kd_tree
[0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5267857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.4708333333333333, 0.5, 0.5125, 0.4976190476190475, 0.4976190476190476]

The optimal number of neighbors is 21.

The accuracy of the knn classifier on Tf-Idf using K_D tree approach for k = 21 is 53.333333%
